### Getting the metagenomic data

Table S5 has all of the metagenomes listed, but it does not provide SRA accession numbers. The only identifying information is a sample number. Most of the data is from the Global Ocean Sampling project from JCVI. Rather than going through one by one and looking up all associated SRAs for each GS number, I'm looking through GenBank BioProjects by searching for sample names matching those in Table S5. The code below is parsing a table of all SRA accessions associated with particular projects, and then parsing the sample numbers from table S5 to identify which ones to download. The random metagenomes not part of GOS will likely be added to the download list manually.

I saved the summary from BioProject 294826 as 'GOS_294826_SraRunTable.txt' to my Desktop and transfered to the wrangling folder within our repo. This file contains all of the metadat of the SRA accessions associated with this BioProject. 

In [5]:
GOS_294826 = [] #initialize list
with open('GOS_294826_SraRunTable.txt','r') as f: #read file
    next(f) #skip header
    for line in f: #loop through lines
        GOS_294826.append(line.strip().split(',')[1].split('_')[2]) #append sample names
GOS_294826[0:5] #print first five sample names

['GS023', 'GS025', 'GS026', 'GS028', 'GS031']

In [6]:
len(GOS_294826) #get length of sample names list

270

Great, this is the format we want.

Looks like there are 270 samples, but I'm pretty sure many of the samples have more than one accession associated with them:

In [7]:
len(set(GOS_294826)) #use set to get only unique names

202

Great. There are 202 unique samples. 

But what if I make a dictionary that maps the sample number to the accession....

The code below should get all of the accessions for all of the samples as long as it is for the 0.1 um fraction.

In [8]:
GOS_294826_dict = {} #initialize empty dictionary
with open('GOS_294826_SraRunTable.txt','r') as f: #open file
    next(f) #skip header
    for line in f: #loop through lines
        if line.strip().split(',')[1].split('_')[3] == '0.1': #if the size fraction in 0.1, proceed (we want to keep these). inferred naming convention indicates size fraction
            samp = line.strip().split(',')[1].split('_')[2] #get SRA accession number
            if samp in GOS_294826_dict.keys(): #if sample already has at least one SRA
                GOS_294826_dict[samp].append(line.strip().split(',')[0]) #append to dictionary value
            else: #if this is the first time sample is seend
                GOS_294826_dict[samp] = [line.strip().split(',')[0]] #create new dictionary value
        else: #if size fraction is not equal to 0.1
            pass  #don't do anything
GOS_294826_dict #print dictionary

{'GS023': ['ERR833296'],
 'GS026': ['ERR833300', 'ERR833299'],
 'GS028': ['ERR833302'],
 'GS031': ['ERR833306'],
 'GS032': ['ERR833307', 'ERR833308'],
 'GS033': ['ERR833310', 'ERR833309'],
 'GS035': ['ERR833312'],
 'GS036': ['ERR833313'],
 'GS038': ['ERR833315'],
 'GS040': ['ERR833317'],
 'GS042': ['ERR833319'],
 'GS047': ['ERR833325'],
 'GS049': ['ERR833329'],
 'GS051': ['ERR833331'],
 'GS053': ['ERR833333'],
 'GS055': ['ERR833335'],
 'GS058': ['ERR833337', 'ERR833339', 'ERR833338', 'ERR833340'],
 'GS061': ['ERR833343'],
 'GS062': ['ERR833344'],
 'GS066': ['ERR833348'],
 'GS069': ['ERR833352', 'ERR833354', 'ERR833351', 'ERR833353'],
 'GS071': ['ERR833356'],
 'GS073': ['ERR833358'],
 'GS075': ['ERR833360'],
 'GS077': ['ERR833362'],
 'GS079': ['ERR833364'],
 'GS082': ['ERR833366'],
 'GS083': ['ERR833368', 'ERR833367'],
 'GS084': ['ERR833370',
  'ERR833371',
  'ERR833373',
  'ERR833369',
  'ERR833372',
  'ERR833374'],
 'GS088': ['ERR833378', 'ERR833379', 'ERR833376', 'ERR833377', 'ERR833

In [9]:
len(GOS_294826_dict.keys()) #print length of dictionary values

199

Okay we've got what we want. A dictionary with 199 keys, one for each sample, pointing to values that are the accession numbers. Now I want to read in Table S5, parse the sample names, and find which ones are not in the GOS_29586_dict.

I used a free online tool to extract the information from the supplental table into a csv. 

In [10]:
Swan_samps = [] #create list to hold all the sample names we want
with open('Table_S5.csv','r') as f: #open the csv
    next(f) #skip the header
    for line in f: #loop over the lines 
        Swan_samps.append(line.strip().split(',')[0].strip('*')) #append sample name to list
Swan_samps[0:5] #print first five

['GS123', 'GS122', 'GS121', 'GS120', 'GS119']

Okay so we've got our list of samples from the paper. Now I want to find which ones are found in the GOS_294826_dict.

In [11]:
set(Swan_samps).intersection(set(GOS_294826_dict.keys())) #find out which of the samples you have already retrieved by looking for intersection between the two sets

{'GS002',
 'GS003',
 'GS004',
 'GS005',
 'GS006',
 'GS007',
 'GS008',
 'GS009',
 'GS010',
 'GS013',
 'GS014',
 'GS015',
 'GS016',
 'GS017',
 'GS018',
 'GS019',
 'GS021',
 'GS022',
 'GS023',
 'GS026',
 'GS027',
 'GS028',
 'GS029',
 'GS030',
 'GS031',
 'GS034',
 'GS035',
 'GS036',
 'GS037',
 'GS048',
 'GS049',
 'GS051',
 'GS108',
 'GS109',
 'GS110',
 'GS111',
 'GS112',
 'GS113',
 'GS114',
 'GS115',
 'GS116',
 'GS117',
 'GS119',
 'GS120',
 'GS121',
 'GS122',
 'GS123',
 'GS148',
 'GS149'}

Okay so here are all of the samples in common. But how many?

In [12]:
len(set(Swan_samps).intersection(set(GOS_294826_dict.keys()))) #find the length of the intersection

49

49 out of the total 115. Not bad. Now I am going to find which don't intersect. Pick some GS numbers and search SRA for the project number. 

In [13]:
set(Swan_samps) - set(GOS_294826_dict.keys()) #find which from the paper are missing from the intersection

{'ECH1_4444_77',
 'ECH2_4444_83',
 'ECH3_4445_65',
 'ECH4_4445_66',
 'ECH5_4445_67',
 'ECH6_4445_68',
 'ECH7_4445_69',
 'ECH8_4445_70',
 'ERS095011',
 'ERS095012',
 'ERS095013',
 'ERS095014',
 'ERS095015',
 'ERS095018',
 'ERS095019',
 'GS000b',
 'GS000c',
 'GS000d',
 'GS001a',
 'GS001b',
 'GS001c',
 'GS025',
 'GS108_454',
 'GS112_454',
 'GS235',
 'GS236',
 'GS346',
 'GS347',
 'GS348',
 'GS349',
 'GS351',
 'GS352',
 'GS353',
 'GS355',
 'GS357',
 'GS358',
 'GS359',
 'GS360',
 'GS362',
 'GS363',
 'GS364',
 'GS366',
 'GS367',
 'GS368',
 'GS369',
 'GS386',
 'GS387',
 'GS388',
 'GS389',
 'GS390',
 'GS391',
 'GS392',
 'GS393',
 'GS394',
 'HF10',
 'HOT215',
 'MED',
 'P12_a',
 'P12_f',
 'P12_j',
 'P26_a',
 'P26_j',
 'P4_a',
 'P4_f',
 'P4_j'}

Apparently there is a duplicate in the Swan_samps. See below:

In [14]:
len(set(Swan_samps)) #print length unique sample names

114

In [15]:
len(Swan_samps) #print length of sample names list

115

The length of the list is one longer than the length of the set (which has no dups). How to find which is the duplicate?

In [16]:
for i in Swan_samps: #loop over elements in sample name list
    if Swan_samps.count(i)>1: #if seen more than once
        print(i+" is the duplicate!") #print which one

GS346 is the duplicate!
GS346 is the duplicate!


Okay so we've found the duplicate. Consulting the table, it looks like this was accidentally added to the table and doesn't actually represent an additional sample. This is substantiated by its absence on Fig. S10. 

So we're off to look for the remaining 64 samples. Starting with other GOS samples. It looks like many of them are in BioProject 50699. The SRA table is in a different format, so will have to modify code from above. There is no information in the metadata for this BioProject to indicate what size fraction these data are from, so I'm just going to have to assume it is 0.1 um.

I downloaded the BioProject summary as I did before and uploaded to poseidon. 

In [17]:
GOS_50699_dict = {} #initialize dictionary
with open('GOS_50699_SraRunTable.txt','r') as f: #open file
    next(f) #skip header
    for line in f: #loop over lines
        samp = "GS"+line.strip().split(',')[3][-3:] #retrieve sample name
        if samp in GOS_50699_dict.keys(): #if already seen
            GOS_50699_dict[samp].append(line.strip().split(',')[0]) #append SRA to dictionary value
        else: #otherwise
            GOS_50699_dict[samp] = [line.strip().split(',')[0]] #assign new value
GOS_50699_dict #print

{'GS377': ['SRR063386', 'SRR062185', 'SRR062246'],
 'GS379': ['SRR063387', 'SRR062187', 'SRR062249'],
 'GS392': ['SRR063388', 'SRR063770', 'SRR062260', 'SRR062261'],
 'GS375': ['SRR063389', 'SRR062243', 'SRR062183'],
 'GS235': ['SRR063767', 'SRR062131', 'SRR062132'],
 'GS391': ['SRR063769', 'SRR062258', 'SRR062259', 'SRR063390'],
 'GS233': ['SRR061742', 'SRR062130', 'SRR062199'],
 'GS394': ['SRR062129', 'SRR062198', 'SRR062264'],
 'GS236': ['SRR062134', 'SRR062201', 'SRR063385'],
 'GS346': ['SRR062135', 'SRR062202', 'SRR062203'],
 'GS347': ['SRR062139', 'SRR062204', 'SRR062205', 'SRR062206'],
 'GS348': ['SRR062140', 'SRR062141'],
 'GS349': ['SRR062142', 'SRR062207', 'SRR062208'],
 'GS350': ['SRR062143', 'SRR062144', 'SRR062209'],
 'GS351': ['SRR062145', 'SRR062146', 'SRR062210', 'SRR062211'],
 'GS352': ['SRR062147', 'SRR062212', 'SRR062213'],
 'GS353': ['SRR062148', 'SRR062214', 'SRR062625'],
 'GS354': ['SRR062149', 'SRR062216', 'SRR062215'],
 'GS355': ['SRR062150', 'SRR062151', 'SRR06

Okay cool so we've got this second dictionary. These were all done with 454 sequencing, so keep that in mind later on when using PRINSEQ (entropy filtering stuff only applies to pyrosequencing...)

Anyway, now I want to merge the two GOS dictionaries and then see which Swan samples are still missing. 

In [ ]:
samp2acc_dict = GOS_294826_dict #copy GOS_294826_dict to new object to map all samples to their SRA accesion number(s)
samp2acc_dict.update(GOS_50699_dict) #update the dictionary with the other GOS dict
samp2acc_dict #print

In [ ]:
set(Swan_samps) - set(samp2acc_dict.keys()) #print those that are still missing

Okay great we now most only have the non-GOS! The ERS* samples are all from one location, listed in Fig. S10 as HI. I think these are from Heligoland? These are represented as one sample, so will be pooled. I am going to manually place these into the dictionary.

In [ ]:
samp2acc_dict['HI'] = ['ERS095011',
 'ERS095012',
 'ERS095013',
 'ERS095014',
 'ERS095015',
 'ERS095018',
 'ERS095019'] #update the samp2acc_dict manually

Now we have all these randos I have to track down. And then there are the lingering GOS...

I was able to track down the ECH samples as coming from:

Gilbert, J. A., Meyer, F., Schriml, L., Joint, I. R., Mühling, M., & Field, D. (2010). Metagenomes and metatranscriptomes from the L4 long-term coastal monitoring station in the Western English Channel. Standards in genomic sciences, 3(2), 183.

This was based just on a Google Scholar search for 'english channel metagenomic 454'. It works...

As a note, I only selected those that were labeled as WGS, not those that were EST. This reduced it to ten samples (greater than 8 because I think for some samples they sequenced the prefilters??) I was able to identify the two odd samples by looking at BioSample metadata. 

I exported from GenBank a list of all associated SRAs into a file called "ECH_acc.txt" and uploaded to Poseidon.

There should be a total of 89 keys in dictionary when you are done (number of columns in Fig. S10)!!!

In [ ]:
with open('ECH_acc.txt','r') as f: #open file
    samp2acc_dict['ECH'] = [] #initialize new key value in sample/accession map
    for line in f: #loop through lines in file
        samp2acc_dict['ECH'].append(line.strip()) #append to value in dictionary
samp2acc_dict['ECH'] #print value of ECH key

Cool. Who's left? The GS000 and GS001 samples aren't in the SRA, apparently. I just found them on iMicrobe and am downloading the sequence files manually. So those don't have to be included in the dictionary. These files do not actually contain the reads but rather have IDs that can be used to capture the desired reads from a huge file with all the reads from this iMicrobe project.

Now I want to add the 454 sequencing for GS108 and GS112. Swan also included the 0.8 and 3um fractions for these samples, as well as for GS048. Adding all now.

In [ ]:
samp2acc_dict['GS108'] #print SRA number already retrieved for this sample

In [ ]:
GOS108_add = ['ERR833396','ERR833397','ERR833398','ERR833399','SRR066138','ERR833403','ERR833402'] #list of SRA numbers to add (found manually)
for i in GOS108_add: #for each element in the list of SRAs to add
    samp2acc_dict['GS108'].append(i) #append them to the dictionary key value
samp2acc_dict['GS108'] #print

In [ ]:
GOS112_add = ['SRR066139','ERR833417','ERR833416','ERR833420','ERR833421'] #same as above
for i in GOS112_add:
    samp2acc_dict['GS112'].append(i)
samp2acc_dict['GS112']

In [ ]:
GOS048_add = ['ERR833327','ERR833328'] #same as above
for i in GOS048_add:
    samp2acc_dict['GS048'].append(i)
samp2acc_dict['GS048']

In [ ]:
samp2acc_dict['GS025'] = ['ERR833298'] #same as above except just one SRA accession to add

Okay now I just need:

 'HF10',
 'HOT215',
 'MED',
 'P12_a',
 'P12_f',
 'P12_j',
 'P26_a',
 'P26_j',
 'P4_a',
 'P4_f',
 'P4_j'
 
Time to sleuth so more.

HF10 and HOT215 are the two Hawaii samples. HOT215 has four matches to BioSamples. Getting 454 data from all four. 

In [ ]:
samp2acc_dict['HOT'] = ["SRR1303811"] #add manually
samp2acc_dict['HOT'] #print

The HF10 data is not in the SRA, but just in GenBank in nucleotide format. Acccessions are DU731018-DU796676 and DU800850-DU800864. This info is at the very end of:

DeLong, E. F., Preston, C. M., Mincer, T., Rich, V., Hallam, S. J., Frigaard, N. U., ... & Chisholm, S. W. (2006). Community genomics among stratified microbial assemblages in the ocean's interior. Science, 311(5760), 496-503.

Will retrieve later. Need to figure out which ones are from surface sample.

Found 'MED' by searching 'mediterranean metagenomics 454'. Published in:

Ghai, R., Martin-Cuadrado, A. B., Molto, A. G., Heredia, I. G., Cabrera, R., Martin, J., ... & Mira, A. (2010). Metagenome of the Mediterranean deep chlorophyll maximum studied by direct and fosmid library 454 pyrosequencing. The ISME journal, 4(9), 1154.

Accession is SRR037008

In [ ]:
samp2acc_dict['MED'] = ['SRR037008'] #add manually

Now I need to get the one that start with P. These are all from around the same area and in Fig. S10 are preceded by NESAP. These also are fosmids. The sequences are on GenBank. Accession numbers KG088956–KG619837. This is from:

Wright, J. J., Mewis, K., Hanson, N. W., Konwar, K. M., Maas, K. R., & Hallam, S. J. (2014). Genomic properties of Marine Group A bacteria indicate a role in the marine sulfur cycle. The ISME journal, 8(2), 455.

Also in some sort of other format (Genome Survey Sequences?) Accession LIBGSS_039072–LIBGSS_039117. 

Will do this later.

Okay! So that should do it. Now I just have to extract the keys that I want from the samp2acc_dict, because a lot of the keys are for samples that aren't included in Swan. 

I just had to type out all of the samples from Fig. S10 so I had them in the right order for making the figure later. I can now use that to pull the accession numbers I need from the samp2acc_dict.

In [ ]:
swan_stations = [] #initialize 
with open('sample_stations.txt','r') as f: #open list of metagenome stations in order
    for line in f: #loop through
        swan_stations.append(line.strip()) #append to list
swan_stations #print

In [ ]:
samp2acc_dict.keys() #print names of keys

In [ ]:
samp2acc_dict_final = {} #initialize final sample to accession map
for i in swan_stations: #loop through
    if i in list(samp2acc_dict.keys()): #if it appears in the original sample to accesion map
        samp2acc_dict_final[i] = samp2acc_dict[i] #update
    else: #otherwise
        pass #do nothing
samp2acc_dict_final #print

In [ ]:
len(list(samp2acc_dict_final.keys())) #get number of samples in final sample accession map. just double checking everything here

In [ ]:
len(swan_stations) #get number of stations from figure

In [ ]:
len(samp2acc_dict) #get number of samples from original sample accession map

In [ ]:
len(samp2acc_dict_final) #get number of samples in final sample accession map

Okay this is what we expect. There are five stations missing. These are GS000 and GS001, because these I downloaded through iMicrobe. The other three are the NESAP P4, P12 and P26. These are fosmids that I need to get from GenBank. I also need to add the fosmid sequences to HOT. 

Okay now I can actually use this dictionary somehow to download all of these files from SRA. I am just going to write this dictionary to a file.

In [ ]:
output = open('acc_dict.txt','w') #write a file that we will pass into a function retrieving the accession we want
for key in samp2acc_dict_final: #for each key in the final sample accesion map 
    output.write(key+'\t'+','.join(map(str,samp2acc_dict_final[key]))+'\n') #write the sample name and then a comma-separated list of all of the associated accessions

Now I am going to work on getting the fosmid sequences. First I'll start with the HF10 from HOT. Acccessions are DU731018-DU796676 and DU800850-DU800864. I want to put these all into a comma separated list for downloading using entrez-direct. 

However, it appears that these accessions refer a lot more sequences than were used in the paper (7,829). There is an identifier in the GenBank record HF10_10-07-02. It is only these samples that we want (I think). Going with just HF10. Saving GenBank record in accession list format. Renamed HF10_acc.txt

Now I can actually use this file to download the sequences. I wrote a SLURM script that loops over this file and uses efetch to get the sequences in fasta format. Saved into a file called HF10.fa in /vortexfs1/omics/env-bio/collaboration/genome-streamlining/data/metagenomes/HOT/fosmid

Next order of business is to get the NESAP data. Then I need to figure out how to extract the reads I want from the GOS_reads that correspond to GS000 and GS001. More on that later. 

Okay it is really hard to figure out which of the fosmid libraries from Wright et al they actually used. I'm going to sit on this for a while and pick up later. 

Using the supplementary table from Wright et al, I looked up the GSS accessions, which are actually BioSamples on NCBI. For each station and month, I pasted the accession number for the 10m depth into NCBI and retrieved an accession list of all associated nucleotides. Each of these were transferred to the cluster, and then they were concatenated together by station. This is almost certainly not the same data that was used in Swan, but it's all we got.

For the NESAP data, I am having a hard time downloading through the command line (really slow), so I am just downloading manually through NCBI and uploading to Poseidon. 

Right so for the GS000 and GS001, I have a master set of reads and then each sample has three .fa files that just have the headers that point to the master reads. I have to figure out some way to grab them. The .fa files are single lines. The way to split them is:

In [26]:
#!sed -i "s/>/\n>/g" file.fa #I was not able to get bash to run in the notebook, so this was done in the command line, as below

The i flag edits the file instead of to STDOUT. So this is going to substitute (the s) every > with a newline then a >. Now it's over multiple lines. The next issue is that the headers don't actually match the master file. They are truncated. So I can probably use awk to grab just the fields I want. Maybe just the first few, including the mate? Or all the way up to where it was truncated? Whatever, I'll do this later.

Okay continuing on. I am going to use the command above to reformat the GS000 and GS001 files. I want to retain just the first 8 fields of the GS001 and GS000 files as well as just the first 8 fields of the GOS reads database. Going to do that with cut. Did it manually for each file and saved to new file with _trunc added to filename. Then deleted the original. COuldn't find an easy way to edit in place with cut.

In [ ]:
#!cut -d ' ' -f1-9 JCVI_SMPL_1103283000002.fa > JCVI_SMPL_1103283000002_trunc.fa

Did this for each of the 6 files. Now I just have the first 8 fields. Doing the same for GOS_reads. First unzipped with gunzip. 

In [ ]:
#!cut -d ' ' -f1-9 GOS_reads.fa > GOS_reads_trunc.fa

Now going to use some sort of grepping to match the input files to the database and pull out the match (header) and the next line (sequence). The -A flag of grep allows you to specify number of lines trailing the match to output. Should be 1 in our case. I think there is also a way of reading in the pattern from a file. Yes that's with the -f flag. This looks through the file, one pattern per line. Going to write a slurm script because this takes a while, but the basic command will look like this (when run from the GS000/1 directories):

In [ ]:
#!grep -f file_trunc.fa -A1 ../GOS_reads/GOS_reads_trunc.fa > outfile

Change of plans. For some reason it isn't working the way I outlined it above. But the GOS_reads headers all have sample names in them (i.e. GS000b, GS001c, etc.) So I am just going to grep for those, printing the matching line and the following and saving output to file. Trial with this on GS001a actually yielded same number of sequences as presented in table s5. So that's good news. 

Example for how I did this below. Did individually for each of the six. 

In [ ]:
#!grep -A1 "GS001b" ../GOS_reads/GOS_reads_trunc.fa > GS001b.fa

All of the numbers of sequence match up perfectly to the table. Now I am going to delete the GOS_reads folder becuase we don't need it anymore